# DeepLearning 03a. Recurrent Neural Nets (RNN)

* **Implementation 4a**: RNN from scratch (basics)
    * *Source*: Adapted from Dan Britz's tutorial on Language Model with RNN
    * *Contribution*:
        * Fixed bugs in the source.
        * Adapted to sequence labeling task.
        * Added annotations.

* **Implementation 4b**: RNN with Keras (basics)
    * *Source*: My RNN code at https://github.com/suwangcompling/texasdataday2017/blob/master/NER_ATIS.ipynb
    * *Contribution*: 
        * Hopefully clearer pipeline

* **Implementation 4c**: RNN with Keras (bidirectional setup + tuning options)
    * *Advanced Model*: Bi-LSTM
    * *Even advanced-ier Model*: Bi-LSTM-CRF (https://github.com/glample/tagger)

## I. Implementation 4a

In [29]:
import sys, nltk
import numpy as np
from nltk.corpus import brown
from itertools import chain
from spacy.en import English
from datetime import datetime
from sklearn.metrics import accuracy_score

In [89]:
# LOAD DATA

sents = brown.sents()

# DATA NORMALIZATION (LEMMATIZATION)

# set limit on vocab size (for demo purpose)
vocabulary_size = 10000
# handle unknown token and corresponding label
unknown_token = "<UNK>"
unknown_label = "UNK"
# start/end tokens (cf. Jurafsky & Martin on n-gram language models)
sentence_start_token = "<S>"
sentence_end_token = "</S>"

# load parser
parser = English()
# lemmatize
X, Y = [], []
for sent in sents:
    words = ' '.join(sent) # list of words -> sent as string, for spacy parser.
    parsed_sent = parser(unicode(words))
    lemmas = [token.lemma_ for token in parsed_sent]
    labels = [token.pos_ for token in parsed_sent] # use spacy's tagging as target.
    X.append(lemmas)
    Y.append(labels)

# build word dictionary for lookup
word_freq = nltk.FreqDist(chain(*X))    
vocab = word_freq.most_common(vocabulary_size - 1) # [(w,freq)...]. leave 1 slot for <UNK>.
i2w = [elem[0] for elem in vocab] + [unknown_token]
w2i = {w:i for i,w in enumerate(i2w)}
# replace words under frequency cut to <UNK>
for i,sent in enumerate(X):
    X[i] = [w if w in w2i else unknown_token for w in sent]
    
# build label dictionary for lookup
label_vocab = list(set(chain(*Y)))
i2l = [x for x in label_vocab] + [unknown_label]
l2i = dict([(l,i) for i,l in enumerate(i2l)])
label_vocabulary_size = len(l2i)
for i,labels in enumerate(Y):
    Y[i] = [l for l in labels]

# one-hot encoding: word/label -> word/label index
train_test_split = (int)(len(X)*0.95)
X_encoded = np.asarray([[w2i[w] for w in sent] for sent in X])
Y_encoded = np.asarray([[l2i[unknown_label] if X[i][j]==unknown_token else l2i[l]
                       for j,l in enumerate(labels)] for i,labels in enumerate(Y)])
X_train = X_encoded[:train_test_split]
Y_train = Y_encoded[:train_test_split]
X_test = X_encoded[train_test_split:]
Y_test = Y_encoded[train_test_split:]

In [101]:
# RNN POS-TAGGING MODEL

def softmax(x):
    """
    Softmax transformation.
    
    Arguments:
    x: List of vocab-index-encoded words.
    
    Returns: Softmax transformed x.
    """
    xt = np.exp(x - np.max(x))
    return xt / np.sum(xt)

class RNN:
    
    def __init__(self, input_dim, output_dim, hidden_dim=100, bptt_truncate=4):
        
        self.input_dim = input_dim
        self.output_dim = output_dim
        self.hidden_dim = hidden_dim
        self.bptt_truncate = bptt_truncate
        
        # input -> hidden projection matrix
        self.U = np.random.uniform(-np.sqrt(1./input_dim), np.sqrt(1./input_dim), (hidden_dim, input_dim))
        # hidden -> output projection matrix
        self.V = np.random.uniform(-np.sqrt(1./hidden_dim), np.sqrt(1./hidden_dim), (output_dim, hidden_dim))
        # hidden (current t) -> hidden (next t) projection matrix
        self.W = np.random.uniform(-np.sqrt(1./hidden_dim), np.sqrt(1./hidden_dim), (hidden_dim, hidden_dim))
        
    def forward_propagation(self, x):
        """
        Feedforward step, feed input through RNN to obtain output.
        
        Arguments:
        x: Input vector, a list of vocab-index-encoded words.
        
        Returns output vector.
        """
        T = len(x)
        s = np.zeros((T+1, self.hidden_dim)) # add 1 more time step for initial state.
        s[-1] = np.zeros(self.hidden_dim) # initialize initial state to 0.
        o = np.zeros((T, self.output_dim))
        # Elman's RNN 
        #   link: https://en.wikipedia.org/wiki/Recurrent_neural_network#Elman_networks_and_Jordan_networks)
        for t in np.arange(T):
            s[t] = np.tanh(self.U[:,x[t]] + self.W.dot(s[t-1]))
            o[t] = softmax(self.V.dot(s[t]))
        return [o, s]
    
    def predict(self, x):
        """
        POS-tag a sentence.
        
        Arguments: 
        x: Input vector, a list of vocab-index-encoded words.
        
        Returns vocab-index-encoded labels.
        """
        o, s = self.forward_propagation(x)
        return np.argmax(o, axis=1)
    
    def calculate_loss(self, x, y):
        """
        Compute cross entropy cost.
        
        Arguments:
        x: Input vector, a list of vocab-index-encoded words.
        y: True label vector, vocab-index-encoded labels.
        
        Returns average cost over N (data size) data.
        """
        N = np.sum((len(y_i) for y_i in y))
        L = 0
        for i in np.arange(len(y)):
            o, s = self.forward_propagation(x[i])
            correct_word_predictions = o[np.arange(len(y[i])), y[i]]
            L += -1 * np.sum(np.log(correct_word_predictions))        
        return L / N

    def bptt(self, x, y):
        """
        Backpropagation Through Time (BPTT) algorithm.
        
        Arguments:
        x: Input vector, a list of vocab-index-encoded words.
        y: True label vector, vocab-index-encoded labels.  
        
        Returns gradients of matrices U, V, W.
        """
        T = len(y)
        o, s = self.forward_propagation(x)
        # empty containers for gradients.
        dLdU = np.zeros(self.U.shape)
        dLdV = np.zeros(self.V.shape)
        dLdW = np.zeros(self.W.shape)
        # cf. Mike Nielsen's deep learning book for detailed derivation (ch3 on backprop).
        delta_o = o
        delta_o[np.arange(len(y)), y] -= 1.
        # for each output backwards...
        for t in np.arange(T)[::-1]:
            # a shortcut for computing gradient (for the more intuitive form, cf. Nielsen book: eq. BP3).
            dLdV += np.outer(delta_o[t], s[t].T)
            # initial delta calculation
            delta_t = self.V.T.dot(delta_o[t]) * (1 - (s[t] ** 2)) # (1 - (s[t] ** 2)): derivative of tanh.
            # backpropagation through time (for at most bptt_truncate steps)
            #   ONLY DIFFERENT FROM STANDARD BACKPROP: accumulate/sum gradients at each time step.
            for bptt_step in np.arange(max(0, t-self.bptt_truncate), t+1)[::-1]:
                dLdW += np.outer(delta_t, s[bptt_step-1])              
                dLdU[:,x[bptt_step]] += delta_t
                delta_t = self.W.T.dot(delta_t) * (1 - s[bptt_step-1] ** 2)
        return [dLdU, dLdV, dLdW]
    
    def update(self, x, y, learning_rate):
        """
        Update weights by gradients.
        
        Arguments:
        x: Input vector, a list of vocab-index-encoded words.
        y: True label vector, vocab-index-encoded labels. 
        """
        dLdU, dLdV, dLdW = self.bptt(x, y)

        self.U -= learning_rate * dLdU
        self.V -= learning_rate * dLdV
        self.W -= learning_rate * dLdW
        
    def evaluate(self, X_test, Y_test):
        """
        Evaluate accuracy.
        
        Arguments:
        X_test: List of lists of vocab-index-encoded words.
        Y_test: List of lists of vocab-index-encoded labels. 
        """
        accuracies = []
        for x,y in zip(X_test, Y_test):
            y_pred = self.predict(x)
            y_true = np.asarray(y)
            accuracies.append(accuracy_score(y_true, y_pred))
        print "Average Accuracy:", np.mean(accuracies)

In [106]:
# STOCHASTIC GRADIENT DESCENT
#   batch_size=1. simpler to batch with Keras (later).

learning_rate = 0.005
epochs = 5
verbose_freq = 1 # frequency of current loss report.

rnn = RNN(vocabulary_size, label_vocabulary_size)

# train to demo on partial data.
X_train_sub = X_train[:5000]
Y_train_sub = Y_train[:5000]

losses = []
num_examples_seen = 0
for epoch in range(epochs):
    if (epoch % verbose_freq == 0):
        loss = rnn.calculate_loss(X_train_sub, Y_train_sub)
        losses.append((num_examples_seen, loss))
        time = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        print "%s: Loss after num_examples_seen=%d epoch=%d: %f" % (time, num_examples_seen, epoch, loss)
        # learning rate decay: adjust the learning rate if loss increases
        if (len(losses) > 1 and losses[-1][1] > losses[-2][1]):
            learning_rate = learning_rate * 0.5 
            print "Setting learning rate to %f" % learning_rate
        sys.stdout.flush()
    # for each training example
    for i in range(len(Y_train_sub)):
        rnn.update(X_train_sub[i], Y_train_sub[i], learning_rate)
        num_examples_seen += 1

# EVALUATION
rnn.evaluate(X_test, Y_test)

2017-02-14 21:54:49: Loss after num_examples_seen=0 epoch=0: 2.772439
2017-02-14 21:55:40: Loss after num_examples_seen=5000 epoch=1: 0.513475
2017-02-14 21:56:31: Loss after num_examples_seen=10000 epoch=2: 0.307118
2017-02-14 21:57:23: Loss after num_examples_seen=15000 epoch=3: 0.230946
2017-02-14 21:58:11: Loss after num_examples_seen=20000 epoch=4: 0.191842
Average Accuracy: 0.911805363531


## II. Implementation 4b

In [75]:
import pickle, os, random
os.environ['KERAS_BACKEND']='tensorflow' 
import numpy as np
from keras.models import Sequential
from keras.layers import Embedding, Activation, TimeDistributed
from keras.layers import Dense, LSTM, Bidirectional, Dropout
from keras.optimizers import Adam
from keras.utils.np_utils import to_categorical

In [54]:
# LOAD DATA

path = "/Users/jacobsw/Desktop/WORK/OJO/NER_PRESENTATION/DATA/atis.pkl"
train_triple, valid_triple, test_triple, dicts = pickle.load(open(path, 'rb'))

X_train, Y_train = train_triple[0], train_triple[2]
X_valid, Y_valid = valid_triple[0], valid_triple[2]
X_test, Y_test = test_triple[0], test_triple[2]

l2i = dicts['labels2idx']
w2i = dicts['words2idx']
i2l = {i:l for l,i in l2i.iteritems()}
i2w = {i:w for w,i in w2i.iteritems()}

In [67]:
# SET CONFIGS

vocab_size = len(w2i)
label_size = len(l2i)
emb_size = 100
hidden_size = 100

num_epochs = 20
valid_freq = 1000
valid_size = 100

In [60]:
def dim_transform_x(x):
    """
    Reshape an x data point to [batch_size, length].
    
    Arguments:
    x: Single x data point.
    
    Returns reshaped data point.
    """
    return np.asarray([x])

def dim_transform_y(y):
    """
    Reshape an y data point to [batch_size, length, label_size] (in binarized representation).
    
    Arguments:
    y: Single y data point.
    
    Returns reshaped data point.
    """
    return to_categorical(np.asarray(y)[:,np.newaxis], nb_classes=label_size)[np.newaxis,:,:]

In [61]:
# TRAINSFORM DATA TO FIT INPUT REQUIREMENTS

X_train, X_valid, X_test = map(dim_transform_x, X_train), map(dim_transform_x, X_valid), map(dim_transform_x, X_test)
Y_train, Y_valid, Y_test = map(dim_transform_y, Y_train), map(dim_transform_y, Y_valid), map(dim_transform_y, Y_test)

In [70]:
# BUILD COMPUTATIONAL GRAPH

rnn = Sequential()
rnn.add(Embedding(input_dim=vocab_size, output_dim=emb_size))
rnn.add(LSTM(output_dim=hidden_size, activation='relu', return_sequences=True))
rnn.add(TimeDistributed(Dense(output_dim=label_size)))
rnn.add(Activation('softmax'))
rnn.compile(loss='categorical_crossentropy', optimizer=Adam(), metrics=['accuracy'])

rnn.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
embedding_8 (Embedding)          (None, None, 100)     57200       embedding_input_8[0][0]          
____________________________________________________________________________________________________
lstm_7 (LSTM)                    (None, None, 100)     80400       embedding_8[0][0]                
____________________________________________________________________________________________________
timedistributed_6 (TimeDistribut (None, None, 127)     12827       lstm_7[0][0]                     
____________________________________________________________________________________________________
activation_6 (Activation)        (None, None, 127)     0           timedistributed_6[0][0]          
Total params: 150427
______________________________________________________________________

In [66]:
# TRAIN

num_iters = 0

for i in xrange(num_epochs):
    for t in xrange(len(X_train)):
        rnn.train_on_batch(X_train[t], Y_train[t])
        num_iters += 1
        if num_iters%valid_freq==0:
            valid_ids = random.sample(range(len(X_valid)), valid_size)
            valid_costs, valid_accs = [], []
            for v in valid_ids:
                valid_cost, valid_acc = rnn.evaluate(X_valid[v], Y_valid[v], verbose=0)
                valid_costs.append(valid_cost); valid_accs.append(valid_acc)
            print "Validation Cost/Accuracy at Iteration", num_iters, ":", np.mean(valid_costs), np.mean(valid_accs) 

# EVALUATE    

test_costs, test_accs = [], []
for e in xrange(len(X_test)):
    test_cost, test_acc = rnn.evaluate(X_test[e], Y_test[e], verbose=0)
    test_costs.append(test_cost); test_accs.append(test_acc)
print "Test Cost/Accuracy:", np.mean(test_costs), np.mean(test_accs)

Validation Cost/Accuracy at Iteration 1000 : 0.630596980916 0.860808443064
Validation Cost/Accuracy at Iteration 2000 : 0.404342949591 0.911917387688
Validation Cost/Accuracy at Iteration 3000 : 0.319398777756 0.9242008791
Validation Cost/Accuracy at Iteration 4000 : 0.241086330384 0.939273726382
Validation Cost/Accuracy at Iteration 5000 : 0.193205771447 0.957866637992
Validation Cost/Accuracy at Iteration 6000 : 0.120419648209 0.971302836597
Validation Cost/Accuracy at Iteration 7000 : 0.143774769793 0.966408751208
Validation Cost/Accuracy at Iteration 8000 : 0.0945306488611 0.976764610591
Validation Cost/Accuracy at Iteration 9000 : 0.0704847894763 0.980819010238
Validation Cost/Accuracy at Iteration 10000 : 0.0993882891076 0.980816777993
Validation Cost/Accuracy at Iteration 11000 : 0.070867968236 0.980805333555
Validation Cost/Accuracy at Iteration 12000 : 0.0725929829256 0.980705998394
Validation Cost/Accuracy at Iteration 13000 : 0.0701369878041 0.978846239016
Validation Cost/Ac

## II. Implementation 4c

* Tuning:
    * Regularization: Dropout
    * Learning: Learning Decay

**NB**: Load data using code in Impl. 4a first. 

**NB**: Providing syntax. Tuning needed to make this thing work.

In [77]:
vocab_size = len(w2i)
label_size = len(l2i)
emb_size = 100
hidden_size = 100

num_epochs = 20
valid_freq = 1000
valid_size = 100

In [81]:
# BUILD COMPUTATIONAL GRAPH

bilstm = Sequential()
bilstm.add(Embedding(input_dim=vocab_size, output_dim=emb_size))
bilstm.add(Bidirectional(LSTM(output_dim=hidden_size, activation='relu', return_sequences=True)))
bilstm.add(Dropout(p=0.5))
bilstm.add(Bidirectional(LSTM(output_dim=hidden_size, activation='relu', return_sequences=True)))
bilstm.add(Dropout(p=0.5))
bilstm.add(TimeDistributed(Dense(output_dim=label_size)))
bilstm.add(Activation('softmax'))

adam = Adam(decay=0.9)
bilstm.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

bilstm.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
embedding_12 (Embedding)         (None, None, 100)     57200       embedding_input_12[0][0]         
____________________________________________________________________________________________________
bidirectional_7 (Bidirectional)  (None, None, 200)     160800      embedding_12[0][0]               
____________________________________________________________________________________________________
dropout_5 (Dropout)              (None, None, 200)     0           bidirectional_7[0][0]            
____________________________________________________________________________________________________
bidirectional_8 (Bidirectional)  (None, None, 200)     240800      dropout_5[0][0]                  
___________________________________________________________________________________________

In [83]:
# TRAIN

num_iters = 0

for i in xrange(num_epochs):
    for t in xrange(len(X_train)):
        bilstm.train_on_batch(X_train[t], Y_train[t])
        num_iters += 1
        if num_iters%valid_freq==0:
            valid_ids = random.sample(range(len(X_valid)), valid_size)
            valid_costs, valid_accs = [], []
            for v in valid_ids:
                valid_cost, valid_acc = bilstm.evaluate(X_valid[v], Y_valid[v], verbose=0)
                valid_costs.append(valid_cost); valid_accs.append(valid_acc)
            print "Validation Cost/Accuracy at Iteration", num_iters, ":", np.mean(valid_costs), np.mean(valid_accs)
    
# EVALUATE    

test_costs, test_accs = [], []
for e in xrange(len(X_test)):
    test_cost, test_acc = bilstm.evaluate(X_test[e], Y_test[e], verbose=0)
    test_costs.append(test_cost); test_accs.append(test_acc)
print "Test Cost/Accuracy:", np.mean(test_costs), np.mean(test_accs)